In [ ]:
# import libraries
import csv
import os
from functools import partial
import json
from math import pi, sqrt, nan

import geopandas as gpd
import matplotlib.pyplot as plt

from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
)
from gerrychain.metrics import (efficiency_gap, 
    mean_median, 
    wasted_votes, 
    polsby_popper,
    partisan_bias,
    partisan_gini
)
from gerrychain.proposals import recom
from gerrychain.updaters import (
    Tally,
    boundary_nodes,
    cut_edges,
    cut_edges_by_part,
    exterior_boundaries,
    interior_boundaries,
    perimeter,
)

from gerrytools.scoring import reock

# create output folder
newdir = "./Outputs/"
os.makedirs(os.path.dirname(newdir + "init.txt"), exist_ok=True)
with open(newdir + "init.txt", "w") as f:
    f.write("Created Folder")

# set paths
graph_path = "./Data/PA_VTDALL.json" 
plot_path = "./Data/VTD_FINAL.shp"

# load geopandas dataframe
df = gpd.read_file(plot_path)
df['GEOID10_copy'] = df['GEOID10']

df = df.set_index('GEOID10_copy')

# helper functions
def num_splits(partition):
    """
    Computes the number of district splits within a given partition.

    Paramters: 
        partition: GerryChain Partition - generated districting plan in current step

    return: 
        int - number of splits
    """

    df["current"] = df[unique_label].map(dict(partition.assignment))
    splits = sum(df.groupby("COUNTYFP10")["current"].nunique() > 1)
    return splits

def custom_eff_gap(total_votes, wasted1, wasted2):
    """
    Computes the efficiency gap score (EGS) for a single district.

    Paramters: 
        total_votes: int - number of total votes in election
        wasted1: int - number of wasted votes for party 1 in election
        wasted2: int - number of wasted votes for party 2 in election

    return: 
        float - EGS value
    """
    return (wasted2 - wasted1) / total_votes

def compute_schwartz(area, perimeter):
    """
    Function that actually computes the Schwartzberg compactness score for a district in a partition.

    Paramters: 
        area: float - area of district
        perimeter: float - perimeter of district

    return: 
        float - Schwartzberg compactness score
    """
    try:
        return 1/(perimeter/(2*pi*sqrt(area/pi)))
    except ZeroDivisionError:
        return nan

def schwartz(partition):
    """
    Compiles Schwartzberg compactness scores for each district in the partition.

    Paramters: 
        partition: GerryChain Partition - generated districting plan in current step

    return: 
        dict - dictionary where the keys are the districts and the values are the schwartzberg scores
    """
    return {
        part: compute_schwartz(
            partition["area"][part], partition["perimeter"][part]
        )
        for part in partition.parts
    }

def flatten(lst):
    """
    Flattens a list of lists to a single list.

    Paramters: 
        lst: list - list of lists to be flattened

    return: 
        list - flattened list
    """
    return [[item] for sublist in lst for item in sublist]

# set unique label
unique_label = "GEOID10"

# define elections
num_elections = 9

election_names = [
    "ATG12",
    "GOV14",
    "GOV10",
    "PRES12",
    "SEN10",
    "ATG16",
    "PRES16",
    "SEN16",
    "SEN12"
]
election_columns = [
    ["ATG12D", "ATG12R"],
    ["F2014GOVD", "F2014GOVR"],
    ["GOV10D", "GOV10R"],
    ["PRES12D", "PRES12R"],
    ["SEN10D", "SEN10R"],
    ["T16ATGD", "T16ATGR"],
    ["T16PRESD", "T16PRESR"],
    ["T16SEND", "T16SENR"],
    ["USS12D", "USS12R"]
]

# create graph object from json file
graph = Graph.from_json(graph_path)
graph.join(df, columns=['geometry'])

# create updaters for Markov chain to track each iteration
updaters = {
    "population": Tally("TOT_POP", alias="population"),
    "cut_edges": cut_edges,
    "non_black_pop": Tally("nBPOP", alias='non_black_pop'),
    'black_pop': Tally('BPOP', alias='black_pop'),
    'area': Tally('areas', alias='area'),
    "perimeter": perimeter,
    "exterior_boundaries": exterior_boundaries,
    "interior_boundaries": interior_boundaries,
    "boundary_nodes": boundary_nodes,
    "cut_edges_by_part": cut_edges_by_part,
    'reock': reock(graph)
}

elections = [
    Election(
        election_names[i],
        {"Democratic": election_columns[i][0], "Republican": election_columns[i][1]},
    )
    for i in range(num_elections)
]

election_updaters = {election.name: election for election in elections}

updaters.update(election_updaters)

# define starting plan/partition 
# select initial plan: '2011_PLA_1', '538CPCT__1', 'REMEDIAL_P'
initial_partition = Partition(graph, 'REMEDIAL_P', updaters)

# set population boundaries for each district
ideal_population = sum(initial_partition["population"].values()) / len(
    initial_partition
)

proposal = partial(
    recom, pop_col="TOT_POP", pop_target=ideal_population, epsilon=0.02, node_repeats=2
)

# set compactness constraint for each district
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]), 4 * len(initial_partition["cut_edges"])
)


# create Markov chain with previously defined parameters/constraints
chain = MarkovChain(
    proposal=proposal,
    constraints=[
        constraints.within_percent_of_ideal_population(initial_partition, 0.02),
        compactness_bound, 
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=10000 # total steps of Markov chain
)

# initialize features and metrics to track during Markov chain
pop_vec = []
wpop_vec = []
bpop_vec = []
cut_vec = []
area_vec = []
perim_vec = []

mms = []
dist_mms = []
egs = []
part_bias = []
part_gini = []
dist_egs = []
dist_polsby = []
dist_reock = []
dist_schwartz = []
hmss = []

splits = []

dist_egs_master = []
dist_mms_master = []

# define chain/loop parameters
t = 0
t_step = 10
dist_loop = False
statewide = True

# color of generated districts
color = 'blue'


# loop through Markov chain
for part in chain:

    t += 1
    # generate/append images/data after every t_step versions of the map
    if t % t_step == 0:
        print(t)

        pop_vec.append(list(part["population"].values()))
        wpop_vec.append(list(part["non_black_pop"].values()))
        bpop_vec.append(list(part["black_pop"].values()))
        cut_vec.append(len(part["cut_edges"]))
        area_vec.append(list(part['area'].values()))
        perim_vec.append(list(part['perimeter'].values()))


        if statewide:

            mms.append([])
            egs.append([])
            hmss.append([])
            part_bias.append([])
            part_gini.append([])

            for elect in range(num_elections):
                mms[-1].append(mean_median(part[election_names[elect]]))
                egs[-1].append(efficiency_gap(part[election_names[elect]]))
                part_bias[-1].append(partisan_bias(part[election_names[elect]]))
                part_gini[-1].append(partisan_gini(part[election_names[elect]]))
                hmss[-1].append(part[election_names[elect]].wins("Democratic"))
        
            # generate and save statewide map
            df["plot" + str(t)] = df.index.map(part.assignment)
            df.plot(column="plot" + str(t), cmap="tab20").axis('off')
            plt.savefig(newdir +'images/statewide/' + "state_plot" + str(t) + ".png")
            plt.close()
            df.drop(columns=["plot" + str(t)], inplace=True)

        # calculate and append compactness metrics
        dist_polsby.append(list(polsby_popper(part).values()))

        dist_schwartz.append(list(schwartz(part).values()))

        dist_reock.append(list(part['reock'].values()))

        # district generation loop
        if dist_loop:

            # do similar thing as statewide, but loop through every district
            dist_egs_master.append([])
            dist_mms_master.append([])
            
            for counter, district in enumerate(part.assignment.parts.keys()): 

                # create dict for specific district in loop and map it to geodataframe 
                dist_geo_dict = {key:val for key, val in part.assignment.items() if val == district}

                df['plot' + str(int(t/t_step)) + str(district)] = df['GEOID10'].map(dist_geo_dict)

                # generate and save each district, set cmap for color
                    # 'Set1' for red 
                    # 'Dark2' for green
                    # 'tab20' for blue
                df.plot(column="plot" + str(int(t/t_step)) + str(district), cmap="tab20").axis('off')
                plt.savefig(newdir + f'images/{color}/' + "plot" + str(int(t/t_step)) + 'd' + str(district) + ".png")
                plt.close()
                df.drop(columns=['plot' + str(int(t/t_step)) + str(district)], inplace=True)
                dist_geo_dict.clear()

                # calculate and append district-level metrics 
                dist_egs.append([])
                dist_mms.append([])

                uid = str(int(t/t_step)) + 'd' + str(district)

                dist_egs[-1].append(uid)
                dist_mms[-1].append(uid)

                # loop through each election to calculate EGS value for each district 
                for elect in range(num_elections):
                    
                    dem_votes = part[election_names[elect]].counts('Democratic')[counter]
                    rep_votes = part[election_names[elect]].counts('Republican')[counter]
                    total_votes = dem_votes + rep_votes

                    wasted_votes_dem, wasted_votes_rep = wasted_votes(dem_votes, rep_votes)

                    dist_egs[-1].append(custom_eff_gap(total_votes, wasted_votes_dem, wasted_votes_rep))
            
                
                dist_egs_master.append(dist_egs[-1])
                dist_mms_master.append(dist_mms[-1])
                
                # reset lists for next iteration
                dist_egs = []
                dist_mms = []

        # save files every 1000 steps in case of memory failure
        if t % 1000 == 0:
            with open(newdir + "master_egs" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(dist_egs_master)

            with open(newdir + "master_polsby" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(flatten(dist_polsby))

            with open(newdir + "master_schwartz" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(flatten(dist_schwartz))

            with open(newdir + "master_reock" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(flatten(dist_reock))

            with open(newdir + "mms" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(mms)

            with open(newdir + "master_mms" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(dist_mms_master)

            with open(newdir + "egs" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(egs)

            with open(newdir + "partisan_bias" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(part_bias)

            with open(newdir + "partisan_gini" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(part_gini)

            with open(newdir + "hmss" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(hmss)

            with open(newdir + "pop" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(flatten(pop_vec))

            with open(newdir + "areas" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(flatten(area_vec))

            with open(newdir + "white_pop" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(flatten(wpop_vec))

            with open(newdir + "black_pop" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(flatten(bpop_vec))

            with open(newdir + "perimeters" + ".csv", "w") as tf1:
                writer = csv.writer(tf1, lineterminator="\n")
                writer.writerows(flatten(perim_vec))


# save files after Markov chain is done
with open(newdir + "master_egs" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(dist_egs_master)

with open(newdir + "master_polsby" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(flatten(dist_polsby))

with open(newdir + "master_schwartz" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(flatten(dist_schwartz))

with open(newdir + "master_reock" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(flatten(dist_reock))

with open(newdir + "mms" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(mms)

with open(newdir + "egs" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(egs)

with open(newdir + "partisan_bias" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(part_bias)

with open(newdir + "partisan_gini" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(part_gini)

with open(newdir + "hmss" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(hmss)

with open(newdir + "pop" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(flatten(pop_vec))

with open(newdir + "areas" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(flatten(area_vec))

with open(newdir + "white_pop" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(flatten(wpop_vec))

with open(newdir + "black_pop" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(flatten(bpop_vec))

with open(newdir + "perimeters" + ".csv", "w") as tf1:
    writer = csv.writer(tf1, lineterminator="\n")
    writer.writerows(flatten(perim_vec))



In [ ]:
"""
Columns in initial geodataframe for reference when selecting/examining available elections, features, and starting districting plans: 

['STATEFP10', 'COUNTYFP10', 'VTDST10', 'GEOID10', 'VTDI10', 'NAME10',
       'NAMELSAD10', 'LSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
       'INTPTLAT10', 'INTPTLON10', 'ATG12D', 'ATG12R', 'GOV10D', 'GOV10R',
       'PRES12D', 'PRES12O', 'PRES12R', 'SEN10D', 'SEN10R', 'T16ATGD',
       'T16ATGR', 'T16PRESD', 'T16PRESOTH', 'T16PRESR', 'T16SEND', 'T16SENR',
       'USS12D', 'USS12R', 'GOV', 'TS', 'HISP_POP', 'TOT_POP', 'WHITE_POP',
       'BLACK_POP', 'NATIVE_POP', 'ASIAN_POP', 'F2014GOVD', 'F2014GOVR',
       '2011_PLA_1', 'REMEDIAL_P', '538CPCT__1', '538DEM_PL', '538GOP_PL',
       '8THGRADE_1', '__ID', 'INT001', 'INT002', 'INT003', 'INT004', 'INT005',
       'INT006', 'INT007', 'INT008', 'LG', 'W1012R', 'W1012D', 'W1016R',
       'W1016D', 'W101216R', 'W101216D', 'W1216R', 'W1216D', 'CD', 'Lower',
       'Upper', 'BPOP', 'nBPOP', 'geometry']
"""


In [ ]:
# order of districts for starting plan - does not change during Markov chain
# for reference in later notebooks
part.assignment.parts.keys()